### Classification approach for exercise

In [1]:
import os,re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics import classification_report,confusion_matrix,log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

pd.set_option('display.max_columns',200)
pd.set_option("display.max_rows",500)

In [2]:
dfTrain = pd.read_csv("exercise_40_train.csv")
dfTest = pd.read_csv("exercise_40_test.csv")
dfTest.head(15)


,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.152540,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.727300,66.416594,28.450466,33.499310,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.629990,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.402660,-0.061040,NaN,64.002500,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.567120
1,1.148654,19.301465,Fri,1.862200,-0.773707,-1.461276,0.0076%,0.443209,0.522113,-1.090886,104.791999,8.805876,1.651993,NaN,-1.396065,99.356609,7.117930,79.061540,$-267.562586413086,5.668347,12.377178,NaN,0.321981,female,2.058123,-0.442525,1107.628399,15.747234,2.027073,NaN,yes,0.608259,North Carolina,55.646392,0.789132,68.881807,32.242558,-1.034680,5-10 miles,11.959804,99910.554923,0.547935,1.001799,1.315020,3.229084,5.873890,49.116516,61.279131,9.360134,1818.390072,2.269700,0.336755,69.832156,2.666487,1.224195,-1.214940,NaN,300.629990,0,July,6.520113,2.101449,-0.871509,NaN,allstate,1.289800,14.402071,7.302161,-1.553879,1.352019,-0.000234,54.104054,9.010198,0.537178,1.489493,-1.106853,ford,44.322947,1.0,6.088895,1.878944,-8.237793,3.636347,3.726452,105.488589,53.387201,-0.751229,0.295234,6.212811,4.876645,-0.848567,7.213829,yes,4.276078,NaN,10.386987,12.527094,1,yes,98.607486
2,4.986860,18.769675,Saturday,1.040845,-1.548690,2.632948,-5e-04%,-1.167885,5.739275,0.222975,102.109546,7.831517,3.055358,2.036434,1.057296,NaN,10.943217,87.567827,$-311.292903116571,3.219583,31.135956,50.048638,1.174485,male,1.609278,-0.303259,1009.911508,12.008190,0.618778,NaN,no,0.680923,NaN,53.178113,0.869321,70.249633,35.207243,48.980294,5-10 miles,14.564732,100729.380783,0.096947,-0.490053,NaN,1.333292,0.750075,48.258898,63.737244,11.564194,1815.680559,1.704048,NaN,67.090400,1.547230,NaN,1.428580,NaN,300.629990,0,January,12.190433,1.793349,-0.114922,48.121885,progressive,-1.755707,14.402071,9.903803,1.720469,0.765756,-0.000234,51.522621,11.700359,-1.867170,0.383319,-1.078648,NaN,48.854080,1.0,8.711055,-0.073306,-8.739095,NaN,2.138771,NaN,49.687134,2.641871,1.718243,NaN,4.455432,1.143388,10.483928,no,2.090868,-1.780474,11.328177,11.628247,0,yes,94.578246
3,3.709183,18.374375,Tuesday,-0.169882,-2.396549,-0.784673,-0.016%,-2.662226,1.548050,0.210141,82.653354,0.436885,1.578106,NaN,-1.287913,102.410965,6.588790,71.825782,$2229.14940030076,7.459929,1.822459,88.144007,0.909556,female,8.864059,0.641209,841.889126,20.904196,0.725017,NaN,no,0.622849,Mississippi,50.311869,0.453211,65.253390,34.432292,52.756665,5-10 miles,18.503815,101476.778846,0.888038,-0.007376,-1.126059,1.129508,-0.455920,44.525657,60.008453,12.852088,2251.680231,2.915405,5.895661,75.219207,NaN,NaN,-0.415800,NaN,300.629990,0,July,6.865209,5.083537,1.685063,46.761738,geico,-0.807993,14.402071,16.576216,0.033036,0.284538,-0.000234,54.625974,13.160347,-0.329204,2.171326,-0.109125,subaru,NaN,1.0,11.742605,-0.253294,-6.641284,4.755348,2.138771,NaN,NaN,1.811825,0.461637,18.198978,3.947223,0.693646,3.862867,no,2.643847,1.662240,10.064961,10.550014,1,NaN,100.346261
4,3.801616,20.205541,Monday,2.092652,-0.732784,-0.703101,0.0186%,0.056422,2.878167,-0.45

### Combine data frames

#### Goal is to encode categorical variables for train & test

In [3]:
dfCombine = pd.concat([dfTrain,dfTest])
dfCombine.iloc[40000:40001,]

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,NaN,4.747627,20.509439,Wednesday,2.299105,-1.815777,-0.752166,0.0098%,-3.240309,0.587948,-0.260721,101.113628,-0.812035,3.251085,-0.004432,-0.917763,102.193597,7.097387,75.256714,$120.216189955777,8.269754,4.794438,56.134458,-0.083641,NaN,5.740955,-3.15254,924.013304,17.697125,1.402273,NaN,no,1.461654,Florida,57.930285,4.7273,66.416594,28.450466,33.49931,5-10 miles,16.776061,99971.844703,0.492812,-0.963616,NaN,NaN,1.266416,53.020326,61.840284,NaN,1825.413159,2.517924,NaN,70.880778,NaN,1.923125,0.239009,NaN,300.62999,0,May,5.461123,5.149473,1.179229,59.346822,progressive,2.219502,17.667792,5.557066,-2.030901,0.636111,-0.000234,62.363381,4.613878,2.415655,3.632041,-0.382482,mercedes,55.938387,1.0,8.325299,-1.274085,-5.663245,2.40266,-0.06104,NaN,64.0025,0.548974,3.420875,11.553023,5.904644,NaN,12.542333,no,3.107683,0.533904,12.438759,7.298306,0,NaN,93.56712


In [4]:
dfCombine.reset_index(drop=True,inplace=True)
dfCombine.tail(1)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
49999,NaN,2.972268,20.485504,Sunday,1.747878,0.197357,1.798676,0.0086%,-0.318751,2.858806,0.08416,99.642148,8.497151,0.878065,-0.338883,0.350812,101.065927,7.900097,76.79105,$-508.049552925335,0.443446,-0.255657,82.889762,-2.112773,female,2.764069,NaN,1028.57299,18.45714,2.473741,-0.578496,yes,0.634841,NaN,50.916587,1.271283,63.837508,29.870539,39.909804,5-10 miles,19.938424,101720.09239,-0.875818,0.765511,NaN,0.996541,-3.489898,49.483851,65.862504,7.303529,1150.95107,1.808276,9.110502,72.228876,-0.61096,2.634673,-1.92103,NaN,300.62999,0,September,6.847741,2.181149,1.297841,50.885163,progressive,0.153248,14.402071,11.865657,-1.29376,0.434589,-0.000234,52.004365,4.865971,NaN,1.524051,0.731896,NaN,45.800511,1.0,7.780886,0.953972,-6.183223,1.565181,3.140183,95.665994,NaN,-0.127836,-1.550846,11.222902,4.651827,1.397517,8.463199,yes,3.942066,1.245289,NaN,11.718582,0,yes,98.307681


### Clean data

#### Removal of null values

In [5]:
dfCombine.isnull().sum()

y       10000
x1          0
x2          0
x3          0
x4          0
x5       3030
x6          0
x7          0
x8          0
x9          0
x10         0
x11      6439
x12         0
x13         0
x14     12292
x15         0
x16     13965
x17         0
x18         0
x19         0
x20         0
x21         0
x22      3000
x23         0
x24      4825
x25         0
x26      3050
x27         0
x28         0
x29         0
x30     40421
x31         0
x32         0
x33      8941
x34         0
x35         0
x36         0
x37         0
x38      2967
x39         0
x40         0
x41     11907
x42     12147
x43         0
x44     42813
x45     10072
x46         0
x47         0
x48         0
x49     16077
x50         0
x51         0
x52     20262
x53         0
x54     15934
x55     22120
x56         0
x57     40541
x58         0
x59         0
x60         0
x61      9059
x62         0
x63      3007
x64      6363
x65         0
x66         0
x67      3045
x68      2984
x69         0
x70         0
x71   

### Drop columns with null values & other un-needed columns

In [6]:
# dfCombine.drop(["x5","x11","x14","x16","x22","x24","x26","x30","x33","x38","x41","x42","x44","x45","x49","x52","x54",\
# "x55","x57","x61","x63","x64","x67","x68","x74","x75","x76","x77","x78","x79","x80","x85","x89","x95","x99",\
# "x83","x86","x88","x91","x92","x94","x96"],axis=1,inplace=True)
# unneeded: x39
dfCombine.drop(["x11","x14","x16","x24","x30","x33","x38","x39","x41","x42","x44","x45","x49","x52","x54",\
"x55","x57","x61","x64","x74","x75","x76","x77","x78","x79","x80","x85","x89","x95","x99",\
"x83","x86","x88","x91","x96"],axis=1,inplace=True)

In [7]:
dfCombine.isnull().sum()

y       10000
x1          0
x2          0
x3          0
x4          0
x5       3030
x6          0
x7          0
x8          0
x9          0
x10         0
x12         0
x13         0
x15         0
x17         0
x18         0
x19         0
x20         0
x21         0
x22      3000
x23         0
x25         0
x26      3050
x27         0
x28         0
x29         0
x31         0
x32         0
x34         0
x35         0
x36         0
x37         0
x40         0
x43         0
x46         0
x47         0
x48         0
x50         0
x51         0
x53         0
x56         0
x58         0
x59         0
x60         0
x62         0
x63      3007
x65         0
x66         0
x67      3045
x68      2984
x69         0
x70         0
x71         0
x72         0
x73         0
x81         0
x82         0
x84         0
x87         0
x90         0
x92      3061
x93         0
x94      2955
x97         0
x98         0
x100        0
dtype: int64

In [8]:
dfCombine[["x5","x22","x26","x63","x67","x68","x92","x94"]].sample(30)

,x5,x22,x26,x63,x67,x68,x92,x94
18980,NaN,61.313371,-0.622774,NaN,14.402071,22.649392,6.333430,3.495313
6168,-0.475327,30.024490,-0.330055,-0.077186,14.402071,19.622046,9.315357,4.642095
30120,NaN,33.542520,-0.842309,-0.008392,NaN,18.916159,5.261618,2.977995
19958,NaN,109.770123,-0.337769,NaN,14.402071,16.989091,7.200700,2.954908
17443,-3.537619,33.188748,-4.425757,0.857456,14.402071,28.031547,5.557540,1.709614
46232,-1.572788,67.600785,0.895919,-1.543204,14.402071,13.304211,9.549205,5.765680
43785,0.639496,79.686199,-0.470341,0.038175,14.402071,17.118425,-0.944080,4.168400
32524,-1.734732,67.366693,-0.370425,-0.445271,14.402071,17.610884,6.328149,NaN
20603,-0.449163,70.633959,-1.815152,1.833388,14.402071,10.791129,0.171589,2.603015
18006,0.711825,49.685039,0.750973,-2.676265,11.499010,19.450227,3.728183,0.591311


### Interpolate missing values

In [9]:
dfCombine.interpolate(method="linear",inplace=True)
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
31781,0.0,6.152333,19.336132,Sun,0.379153,1.692065,-0.689577,0.0021%,-0.450959,4.253360,0.654724,7.113846,7.222348,-0.755193,6.723053,83.447874,$269.420345556535,6.409090,5.158188,61.303621,0.355937,4.171408,1.762158,839.468878,16.217819,1.267417,no,1.676888,47.626859,2.224466,70.359251,31.317545,18.192206,1.396232,-2.164955,50.908829,56.753137,2812.124855,-1.714621,70.804066,0.362750,300.629990,0,July,7.647130,0.363097,allstate,-0.177999,14.402071,12.547840,0.419717,0.567567,-0.000234,43.169597,3.165063,-2.499646,-5.800961,2.138771,-1.526868,3.834724,6.345175,no,4.586587,10.746317,0,99.494923
12473,0.0,0.907120,24.017640,Saturday,0.779066,-1.929447,2.424314,0.0043%,-2.353797,1.100592,0.555853,2.011547,4.981047,-0.031826,5.438583,75.236144,$-164.237666772927,-0.089014,1.702695,69.585838,-1.342109,3.106428,-3.035536,824.028324,21.133726,-1.097736,no,0.340222,50.374201,4.571046,65.467921,23.367302,18.462088,2.859582,-1.972753,42.806791,62.967156,2497.698699,-0.743943,67.956898,0.567188,300.629990,0,January,5.883301,0.969795,esurance,0.640724,14.402071,17.930475,0.280554,0.306825,-0.000234,66.992970,4.307929,-2.527803,-3.823880,2.138771,-0.489162,1.508240,6.364788,no,5.236990,7.530791,1,90.605601
19900,0.0,-0.123385,19.504144,Monday,-2.695419,0.796772,0.652831,0.0085%,-1.775996,2.614620,2.106164,6.453064,2.260257,0.930211,5.624118,72.703994,$-1404.57680080601,5.963909,0.828335,37.485618,0.370243,1.522024,1.190385,1047.687075,15.574762,1.072493,no,-0.678757,47.985101,1.186611,65.721378,31.918426,19.986548,1.259602,-0.771899,53.179090,64.088272,586.162759,2.095000,67.052621,0.071903,147.124053,0,December,0.789843,-0.131811,allstate,-2.204570,14.402071,10.019005,0.770916,0.539091,-0.000234,53.999189,3.644677,2.369272,-9.124064,2.138771,1.640887,2.836526,0.565637,no,4.278527,12.383499,1,98.672937
19201,0.0,0.117723,17.239438,Tuesday,-1.406980,-0.107757,0.111944,-0.0029%,-0.914733,4.041597,0.738707,0.202707,5.625008,-1.123931,7.833700,71.819362,$329.618242585919,8.650694,7.798425,81.640302,0.931487,4.273796,-2.051147,1002.626307,15.265990,0.008672,no,-0.274554,51.439352,3.152717,66.871637,35.601146,18.020569,0.581905,0.563027,49.536871,46.715540,1776.941982,-0.521485,69.344718,-1.677587,300.629990,0,June,2.743368,2.193796,progressive,-1.280103,14.402071,16.420235,-0.719141,0.524706,-0.000234,51.181243,4.442724,-0.794173,-4.155530,2.138771,-1.369854,3.271945,3.367526,no,3.220641,7.040162,1,97.838983
39179,0.0,4.704678,18.148591,Thursday,-0.888025,2.504513,-0.051067,0.0055%,0.514483,3.580039,0.402596,2.508419,3.330504,0.308413,6.946737,84.746718,$883.000232029013,8.584346,14.547694,47.747082,1.403208,3.919480,3.550688,976.166639,13.439135,2.333772,no,-1.227843,50.708488,1.547302,72.604177,32.383506,17.675447,0.975642,-0.011010,51.244436,55.284700,3032.448673,2.698971,64.340907,1.418018,300.629990,0,December,3.905976,1.034600,allstate,-1.851973,14.402071,12.644999,1.602516,0.852048,1.599886,47.208325,5.608313,-0.941709,-7.303543,4.735399,0.164846,3.274648,4.984331,no,4.873530,9.839634,0,97.305314
40631,0.0,1.487025,19.057590,Tuesday,-1.378951,0.933801,0.359624,-0.005%,2.372394,3.448572,0.056224,-0.346889,4.903374,-0.114476,10.231810,79.236872,$53.3895607130214,8.007570,50.990456,70.045778,0.239132,3.182896,-0.148609,906.072106,14.294429,-1.396230,no,2.589203,60.799510,2.390981,70.176562,33.000789,12.247006,0.142762,0.920810,48.885577,54.129425,3322.230324,2.555264,64.183702,1.696790,300.629990,0,August,2.724803,-0.741135,progressive,-1.070258,14.402071,15.082293,1.654511,1.234596,-0.000234,35.595015,3.858270,-0.877562,-4.709336,2.138771,-0.072918,3.808288,6.079685,no,2.789137,7.401835,1,95.676367
9795,0.0,4.988083,16.170532,Fri,-0.780454,2.075966,0.130068,0.0193%,-1.960786,1.643604,-2.16

In [10]:
dfCombine.isnull().sum()

y       0
x1      0
x2      0
x3      0
x4      0
x5      0
x6      0
x7      0
x8      0
x9      0
x10     0
x12     0
x13     0
x15     0
x17     0
x18     0
x19     0
x20     0
x21     0
x22     0
x23     0
x25     0
x26     0
x27     0
x28     0
x29     0
x31     0
x32     0
x34     0
x35     0
x36     0
x37     0
x40     0
x43     0
x46     0
x47     0
x48     0
x50     0
x51     0
x53     0
x56     0
x58     0
x59     0
x60     0
x62     0
x63     0
x65     0
x66     0
x67     0
x68     0
x69     0
x70     0
x71     0
x72     0
x73     0
x81     0
x82     0
x84     0
x87     0
x90     0
x92     0
x93     0
x94     0
x97     0
x98     0
x100    0
dtype: int64

### x3
Clean up day names

In [11]:
dfCombine["x3"].unique()

array(['Wed', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Saturday',
       'Sat', 'Wednesday', 'Sun', 'Tue', 'Thur', 'Monday', 'Fri', 'Mon'],
      dtype=object)

In [12]:
dfCombine["x3"] = dfCombine["x3"].replace(["Mon"],"Monday")
dfCombine["x3"] = dfCombine["x3"].replace(["Tue"],"Tuesday")
dfCombine["x3"] = dfCombine["x3"].replace(["Wed"],"Wednesday")
dfCombine["x3"] = dfCombine["x3"].replace(["Thur"],"Thursday")
dfCombine["x3"] = dfCombine["x3"].replace(["Fri"],"Friday")
dfCombine["x3"] = dfCombine["x3"].replace(["Sat"],"Saturday")
dfCombine["x3"] = dfCombine["x3"].replace(["Sun"],"Sunday")

In [13]:
dfCombine["x3"].unique()

array(['Wednesday', 'Friday', 'Thursday', 'Tuesday', 'Sunday', 'Saturday',
       'Monday'], dtype=object)

### x7
Remove percent sign

In [14]:
dfCombine['x7'] = dfCombine["x7"].str.replace("%","")

In [15]:
dfCombine.head(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,$-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,$-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-8e-04,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,$-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,$-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,$967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935


In [16]:
# remove "E" notation
dfCombine["x7"] = pd.to_numeric(dfCombine["x7"])
dfCombine.sample(25)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
4010,0.0,2.367744,20.529628,Thursday,-0.192869,2.004715,-0.859394,-0.0165,1.495263,-0.416752,-0.001031,-0.202942,3.912375,0.594421,5.245484,86.882777,$1536.17720672403,9.396672,6.194135,76.493262,1.892143,8.290780,0.274073,1121.379367,11.842674,0.467448,no,0.269007,53.861601,1.671240,77.202561,26.688689,10.552583,-0.970738,-1.945308,50.489246,60.249766,3290.895613,-0.005633,62.741940,1.315692,300.629990,0,July,4.510955,-0.876118,allstate,1.491691,14.402071,20.742818,-0.036277,1.164666,-0.000234,57.030777,1.237359,-0.568739,-6.433402,2.138771,-0.284609,3.041477,8.930963,no,6.158661,7.860261,0,96.990692
17304,0.0,2.157367,17.415653,Thursday,-0.431223,1.312507,-1.212351,-0.0094,0.373995,4.536326,-1.394070,2.877613,3.259838,2.871844,9.409296,89.382065,$729.104341127935,4.829143,0.636189,40.299876,0.388596,1.097606,-0.626583,1130.308734,12.016057,-0.591462,yes,0.456558,53.624429,-0.982898,74.517940,31.149248,16.425687,2.588507,-0.846972,52.423242,55.935103,2546.167437,-0.765283,73.885161,-0.068766,300.629990,0,December,4.388896,1.121772,allstate,0.828714,14.402071,17.136867,0.235681,0.727866,0.213905,40.970523,4.332880,0.473753,-8.332454,4.056183,-0.848942,4.724482,4.869075,yes,3.159881,9.821771,0,105.112553
34087,0.0,5.804372,17.057679,Wednesday,-0.695811,0.891840,-1.278062,-0.0264,0.721206,2.192405,-1.764370,0.326147,6.466948,-1.075984,10.737885,89.743139,$593.416478692161,10.085833,1.009584,91.469651,3.085416,3.568812,3.123844,1006.222050,11.192367,2.166586,no,-1.154152,58.865256,2.222552,74.759509,34.472231,13.282726,0.416648,-0.181046,57.971482,57.385589,4377.683855,-0.748832,67.968526,1.799332,300.629990,0,January,10.181893,0.368689,allstate,-1.863247,14.402071,13.296741,0.370985,0.719475,-0.000234,40.666825,4.991154,-4.887888,-8.874535,2.138771,-1.122329,2.475032,11.046533,no,5.377964,8.397126,1,106.782488
10385,1.0,1.686810,20.422436,Wednesday,-2.160301,0.365605,1.189986,-0.0082,-1.300610,3.599191,0.305015,7.630414,5.491448,2.055179,11.236559,82.648964,$1785.32695144464,-0.400240,1.031346,55.275609,-0.916971,3.501797,0.477372,977.312434,11.645360,1.186701,no,1.311851,50.604309,3.853535,74.845891,29.554745,14.344436,1.758023,-0.397589,56.234118,60.613188,2316.666682,0.114429,61.862591,1.674337,300.629990,0,January,6.641793,0.025361,allstate,1.538339,6.619781,15.535537,0.242316,0.433374,-0.000234,41.992960,4.901104,0.532513,-6.758924,2.138771,-0.171147,1.541956,9.077386,no,6.273712,12.473553,1,104.111734
49007,0.0,3.365842,21.373347,Monday,1.552885,-2.049815,-0.116837,0.0007,1.648754,3.739607,-0.241030,8.716513,5.000246,2.090907,8.800188,73.752417,$912.116678547103,1.666680,19.659439,57.420578,-0.000530,3.250351,0.733080,1073.726195,16.018768,0.974681,no,-1.487275,46.928595,1.198303,64.824548,35.324733,14.019896,0.935880,-0.466763,47.801871,56.372909,2508.754131,1.555820,69.566740,-0.548943,300.629990,0,July,-0.132164,0.541395,allstate,-1.071567,14.494474,17.225922,-0.040375,1.133157,-0.000234,40.407973,8.341913,0.243405,-0.459391,2.138771,0.680807,2.028965,5.969596,no,4.481659,9.152530,0,101.798337
6506,1.0,2.703711,17.590990,Friday,-0.044775,0.087664,1.777662,-0.0003,0.344275,3.981359,0.813611,6.514089,4.656391,0.915386,11.010934,82.663904,$-246.034702070151,0.692100,25.191946,43.167714,0.627735,2.364520,1.354075,1321.551515,12.372012,4.995986,no,-0.032491,49.456817,0.977861,69.452831,30.147909,12.212421,0.479865,-0.316063,50.988022,63.655585,1023.080186,1.516863,69.571005,-0.580180,300.629990,1,July,-1.122397,0.880774,esurance,-2.626278,14.402071,10.571509,0.595817,0.867666,-0.000234,49.809351,7.831145,-1.991337,-5.194779,2.138771,-0.901113,4.911799,5.728898,no,3.997413,13.716451,1,99.617471
37093,0.0,2.797962,23.005770,Monday,2.115252,3.051937,-1.570434,-0.0089,-0.505494,

### x19

In [17]:
# replace dollar sign
dfCombine["x19"] = dfCombine["x19"].replace(to_replace="[\$]",value="",regex=True)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758424405,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.9622875143,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187402955527,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626380634258,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007090837503,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163475698959,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.6

### Round to 6 decimal places

In [18]:
dfCombine["x19"] = dfCombine["x19"].astype(float).round(6)
dfCombine.head(10)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,no,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,no,5.492487,7.627730,0,104.251338
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,no,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,no,3.346429,10.505284,1,101.230645
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,no,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,no,4.456565,7.810979,0,109.345215
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,no,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,no,4.100022,9.178325,1,103.021970
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,yes,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,yes,3.234033,11.702664,0,92.925935
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,no,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,no,4.183036,5.632881,0,97.129172
6,0.0,0.094162,18.894755,Thursday,2.027918,1.580408,1.190932,0.0174,-0.331730,2.178622,-0.454172,1.762009,2.125462,-0.907832,8.631917,71.282134,749.965010,14.2208

### x31

In [19]:
# map 0 to no and 1 to yes
dfCombine["x31"] = dfCombine["x31"].map({"no":0,"yes":1})
dfCombine["x31"].unique()

array([0, 1], dtype=int64)

### x93

In [20]:
dfCombine["x93"] = dfCombine["x93"].map({"no":0,"yes":1})
dfCombine["x93"].unique()

array([0, 1], dtype=int64)

### Encode categorical variables

### x3

In [21]:
# days of the week
x3 = pd.get_dummies(dfCombine['x3'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x3,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,0,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.298318,1.670786,67.591932,0.493780,300.629990,0,August,4.106257,-2.246804,progressive,-0.065794,14.402071,23.446751,-2.378431,1.393887,-0.000234,49.858508,18.739751,1.887206,-3.892529,2.138771,1.027475,1.682037,5.253010,0,4.183036,5.632881,0,97.129172,0,0,1,0,0,0,0


#### x60

In [22]:
# months
x60 = pd.get_dummies(dfCombine['x60'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x60,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday,cat__April,cat__August,cat__December,cat__February,cat__January,cat__July,cat__June,cat__March,cat__May,cat__November,cat__October,cat__September
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.139008,2.233107,0,0.488196,45.559596,0.359074,76.835865,37.602918,12.628248,1.452704,-1.573853,52.359238,46.900080,2959.

#### x65

In [23]:
# insurance name
x65 = pd.get_dummies(dfCombine['x65'],prefix='cat_')
dfCombine = pd.merge(
    left=dfCombine,
    right=x65,
    left_index=True,
    right_index=True,
)

dfCombine.head(15)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x12,x13,x15,x17,x18,x19,x20,x21,x22,x23,x25,x26,x27,x28,x29,x31,x32,x34,x35,x36,x37,x40,x43,x46,x47,x48,x50,x51,x53,x56,x58,x59,x60,x62,x63,x65,x66,x67,x68,x69,x70,x71,x72,x73,x81,x82,x84,x87,x90,x92,x93,x94,x97,x98,x100,cat__Friday,cat__Monday,cat__Saturday,cat__Sunday,cat__Thursday,cat__Tuesday,cat__Wednesday,cat__April,cat__August,cat__December,cat__February,cat__January,cat__July,cat__June,cat__March,cat__May,cat__November,cat__October,cat__September,cat__allstate,cat__esurance,cat__farmers,cat__geico,cat__progressive
0,0.0,0.165254,18.060003,Wednesday,1.077380,-1.339233,-1.584341,0.0062,0.220784,1.816481,1.171788,4.644568,4.814885,-0.587361,4.278444,77.709700,-908.650758,7.328029,36.601967,126.384294,1.264713,3.834082,2.352406,905.491445,14.779467,0.752524,0,-0.597288,46.667221,3.159002,69.444841,32.450076,18.129499,1.227266,0.234654,53.359113,58.251700,1602.756464,2.740134,72.167186,-1.532562,300.629990,0,August,6.600008,1.252479,farmers,1.212048,12.896733,15.263491,0.518653,0.543768,-0.912506,53.521544,7.723719,-0.471594,-3.261865,0.383351,-0.162668,2.770844,0.909479,0,5.492487,7.627730,0,104.251338,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,1.0,2.441471,18.416307,Friday,1.482586,0.920817,-0.759931,0.0064,1.192441,3.513950,1.419900,1.459868,1.443983,-1.165869,8.487132,70.808566,-1864.962288,6.796408,3.789176,93.037021,0.346201,4.345028,1.651579,868.784447,25.914680,1.445294,0,1.127376,47.928569,-1.013712,61.289132,35.521824,11.389081,-0.822055,1.104631,37.728834,56.052749,3350.837875,1.995635,73.964560,0.956001,300.629990,0,April,3.864254,0.057936,allstate,0.738526,14.402071,33.940951,-0.140384,1.016703,-0.000234,46.797571,7.260365,1.691428,-4.789183,2.138771,1.252022,3.703013,9.093466,0,3.346429,10.505284,1,101.230645,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1.0,4.427278,19.188092,Thursday,0.145652,0.366093,0.709962,-0.0008,0.952323,0.782974,-1.247022,1.098525,1.216059,0.211685,8.601193,75.922820,-543.187403,3.802395,7.407996,71.022413,0.070968,4.332644,-0.375737,1077.233497,13.177479,4.174536,0,2.605279,56.674425,0.108486,69.798923,30.684074,14.372443,0.150728,-2.561083,50.236892,63.975108,3756.910196,1.772648,59.428690,4.127857,182.369349,0,September,1.370661,-0.239705,geico,-0.032980,14.402071,18.211817,-0.819798,1.010811,-0.000234,48.202036,9.336021,-2.608974,-6.387984,2.138771,0.657292,3.997183,0.305170,0,4.456565,7.810979,0,109.345215,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
3,0.0,3.925235,19.901257,Tuesday,1.763602,-0.251926,-0.827461,-0.0057,-0.520756,1.825586,2.223038,-1.390239,3.962961,-2.046856,6.611554,74.966925,-182.626381,7.728963,8.136213,121.610846,0.700954,7.294990,-0.603983,1051.655489,17.006528,2.347355,0,1.071202,59.154933,1.319711,65.408246,34.401290,13.191173,-0.965711,-2.123048,41.857197,59.226119,1961.609788,3.155214,68.671023,0.663759,300.629990,0,September,0.013162,0.318335,geico,-0.466535,14.402071,14.629914,1.389325,0.704880,-1.510949,49.882647,5.661421,1.424316,-5.431998,2.138771,2.066346,2.933707,5.971782,0,4.100022,9.178325,1,103.021970,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.0,2.868802,22.202473,Sunday,3.405119,0.083162,1.381504,0.0109,-0.732739,2.151990,-0.275406,7.230125,3.877312,-1.201565,9.103015,77.977156,967.007091,2.272765,24.452102,56.492317,-1.102387,6.313827,0.429187,949.904947,16.962710,0.510985,1,2.283921,46.445617,0.022747,66.662910,31.135261,17.210430,1.012093,3.477451,47.844153,55.921988,2345.195505,3.253079,71.853326,-2.273628,149.725023,0,January,1.305936,0.418272,geico,-0.195764,14.402071,12.227512,-2.951163,1.096548,-0.000234,51.349106,9.422401,0.705003,-5.664815,2.138771,0.663113,2.922302,1.046096,1,3.234033,11.702664,0,92.925935,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
5,0.0,1.926053,19.524376,Saturday,3.702574,2.300792,-0.331106,0.0079,3.085443,0.472701,1.943171,-3.582885,3.522031,-2.799700,7.283226,87.734440,271.163476,15.770180,44.874132,45.729121,0.320526,2.245217,4.303686,1187.333248,8.1

In [24]:
dfTrainClean = dfCombine.iloc[:40000,]
dfTestClean = dfCombine.iloc[40000:,]

### Logistic regression (classification) & neural network

class sklearn.linear_model.LogisticRegression(penalty='l2',dual=False,tol=0.0001,C=1.0,fit_intercept=True,
intercept_scaling=1,class_weight=None,random_state=None,solver='lbfgs',max_iter=100,multi_class='auto', 
verbose=0,warm_start=False,n_jobs=None,l1_ratio=None)

class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100, ), activation='relu', *, 
solver='adam',alpha=0.0001,batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
power_t=0.5,max_iter=200,shuffle=True, random_state=None, tol=0.0001, verbose=False, 
warm_start=False, momentum=0.9,nesterovs_momentum=True, early_stopping=False, 
validation_fraction=0.1,beta_1=0.9,beta_2=0.999,epsilon=1e-08,n_iter_no_change=10, max_fun=15000)

In [25]:
dfCombine.columns

Index(['y', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x12',
       'x13', 'x15', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x25',
       'x26', 'x27', 'x28', 'x29', 'x31', 'x32', 'x34', 'x35', 'x36', 'x37',
       'x40', 'x43', 'x46', 'x47', 'x48', 'x50', 'x51', 'x53', 'x56', 'x58',
       'x59', 'x60', 'x62', 'x63', 'x65', 'x66', 'x67', 'x68', 'x69', 'x70',
       'x71', 'x72', 'x73', 'x81', 'x82', 'x84', 'x87', 'x90', 'x92', 'x93',
       'x94', 'x97', 'x98', 'x100', 'cat__Friday', 'cat__Monday',
       'cat__Saturday', 'cat__Sunday', 'cat__Thursday', 'cat__Tuesday',
       'cat__Wednesday', 'cat__April', 'cat__August', 'cat__December',
       'cat__February', 'cat__January', 'cat__July', 'cat__June', 'cat__March',
       'cat__May', 'cat__November', 'cat__October', 'cat__September',
       'cat__allstate', 'cat__esurance', 'cat__farmers', 'cat__geico',
       'cat__progressive'],
      dtype='object')

In [26]:
features = [
    'x1','x2','x4','x5','x6','x7','x8','x9','x10','x12',
    'x13','x15','x17','x18', 'x19', 'x20', 'x21','x22', 'x23', 'x25',
    'x26','x27','x28','x29', 'x31', 'x32', 'x34','x35', 'x36', 'x37',
    'x40','x43','x46','x47', 'x48', 'x50', 'x51','x53', 'x56', 'x58',
    'x59','x62','x63','x66', 'x67', 'x68', 'x69','x70',
    'x71','x72','x73','x81', 'x82', 'x84', 'x87','x90','x92','x93',
    'x94','x97','x98', 'x100', 'cat__Friday', 'cat__Monday',
    'cat__Saturday','cat__Sunday','cat__Thursday', 'cat__Tuesday',
    'cat__Wednesday','cat__April','cat__August','cat__December',
    'cat__February','cat__January','cat__July','cat__June', 'cat__March',
    'cat__May','cat__November', 'cat__October', 'cat__September',
    'cat__allstate','cat__esurance', 'cat__farmers', 'cat__geico',
    'cat__progressive']

# x4 x6 x9 x10 x12 x17 x22 x23 x34 x48 x53 x59 x69 x73 x81 x92 x97
dropFeatures = [
    'x1','x2','x5','x7','x8',
    'x13','x15','x18','x19', 'x20','x21','x25',
    'x26','x27','x28','x29', 'x31','x32','x35', 'x36', 'x37',
    'x40','x43','x46','x47', 'x50','x51', 'x56','x58',
    'x62','x63','x66', 'x67', 'x68','x70',
    'x71','x72','x82','x84', 'x87','x90','x93',
    'x94','x98', 'x100', 'cat__Friday', 'cat__Monday',
    'cat__Saturday','cat__Sunday','cat__Thursday', 'cat__Tuesday',
    'cat__Wednesday','cat__April','cat__August','cat__December',
    'cat__February','cat__January','cat__July','cat__June', 'cat__March',
    'cat__May','cat__November', 'cat__October', 'cat__September',
    'cat__allstate','cat__esurance', 'cat__farmers', 'cat__geico',
    'cat__progressive']

### Scale data

In [27]:
scalerX = StandardScaler()
scalerX.fit(dfTrainClean[dropFeatures])
xTrainScaled = scalerX.transform(dfTrainClean[dropFeatures])
xTestScaled = scalerX.transform(dfTestClean[dropFeatures])

In [28]:
# logistic regression
logReg = LogisticRegression(C=7.7777,solver="lbfgs",max_iter=1111,tol=0.000003333, \
    multi_class="ovr",class_weight=None)

logReg.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

LogisticRegression(C=7.7777, max_iter=1111, multi_class='ovr', tol=3.333e-06)

In [29]:
# neural networks
mlp = MLPClassifier(hidden_layer_sizes=(76,76,),random_state=None,alpha=0.1111, \
    solver='sgd',learning_rate="adaptive",activation="relu",n_iter_no_change=10, \
    tol=0.00001111,learning_rate_init=0.001111,max_iter=253,momentum=0.9111, \
    batch_size="auto",shuffle=True,warm_start=False,early_stopping=False,validation_fraction=0.1)

mlp.fit(xTrainScaled.astype(float),dfTrainClean["y"].astype(int))

C:\Users\TamLe\anaconda3\envs\sports\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (253) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=0.1111, hidden_layer_sizes=(76, 76),
              learning_rate='adaptive', learning_rate_init=0.001111,
              max_iter=253, momentum=0.9111, solver='sgd', tol=1.111e-05)

### Predictions

In [30]:
yPredTrainLog = logReg.predict(xTrainScaled.astype(float))
yPredTestLog = logReg.predict(xTestScaled.astype(float))
yProbTrainLog = logReg.predict_proba(xTrainScaled.astype(float))
yProbTestLog = logReg.predict_proba(xTestScaled.astype(float))

In [43]:
yPredTrainNN = mlp.predict(xTrainScaled.astype(float))
yPredTestNN = mlp.predict(xTestScaled.astype(float))
yProbTrainNN = mlp.predict_proba(xTrainScaled.astype(float))
yProbTestNN = mlp.predict_proba(xTestScaled.astype(float))

In [44]:
pd.DataFrame(yPredTrainLog).value_counts()

0    39238
1      762
dtype: int64

### Score

In [34]:
# logistic regression
logisticAccuracy = accuracy_score(dfTrainClean["y"].astype(int),yPredTrainLog.astype(int))
print("accuracy of training set: %s" % accuracy)


accuracy of training set: 0.858725


In [46]:
# neural networks
neuralNetworksAccuracy = accuracy_score(dfTrainClean["y"].astype(int),yPredTrainNN.astype(int))
print("accuracy of training set: %s" % neuralNetworksAccuracy)

accuracy of training set: 0.871125


In [47]:
pd.DataFrame(dfTrainClean["y"]).value_counts()

y  
0.0    34197
1.0     5803
dtype: int64

In [38]:
pd.DataFrame(yPredTrainLog).value_counts()

0    39238
1      762
dtype: int64

In [48]:
pd.DataFrame(yPredTrainNN).value_counts()

0    38414
1     1586
dtype: int64

### Probabilities on training set

In [49]:
pd.DataFrame(yProbTrainLog).head(20)

,0,1
0,0.959971,0.040029
1,0.658801,0.341199
2,0.828818,0.171182
3,0.549976,0.450024
4,0.983968,0.016032
5,0.888512,0.111488
6,0.958273,0.041727
7,0.859887,0.140113
8,0.924804,0.075196
9,0.854813,0.145187


In [50]:
pd.DataFrame(yProbTestLog).tail(25)

,0,1
9975,0.946557,0.053443
9976,0.909108,0.090892
9977,0.808916,0.191084
9978,0.656960,0.343040
9979,0.330718,0.669282
9980,0.838664,0.161336
9981,0.948758,0.051242
9982,0.948408,0.051592
9983,0.943232,0.056768
9984,0.768390,0.231610


In [51]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].head(15)

,1
0,0.020053
1,0.027552
2,0.259752
3,0.177349
4,0.013140
5,0.057507
6,0.274311
7,0.059734
8,0.326363
9,0.623363


### Save results

In [52]:
pd.DataFrame(yProbTestLog).iloc[:,[1]].to_csv("glmresults.csv",header=False,index=False)
pd.DataFrame(yProbTestNN).iloc[:,[1]].to_csv("nonglmresults.csv",header=False,index=False)

In [53]:
print(len(yProbTestNN))

10000


### Statsmodels

In [58]:
xInclConstant = sm.add_constant(dfTrainClean[dropFeatures])
model = sm.Logit(dfTrainClean["y"].astype(int),xInclConstant.astype(float))
# logistic regression
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.359918
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                      y   No. Observations:                40000
Model:                          Logit   Df Residuals:                    39933
Method:                           MLE   Df Model:                           66
Date:                Fri, 11 Jun 2021   Pseudo R-squ.:                  0.1308
Time:                        09:52:49   Log-Likelihood:                -14397.
converged:                       True   LL-Null:                       -16563.
Covariance Type:            nonrobust   LLR p-value:                     0.000
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const                3.5752   2.06e+06   1.73e-06      1.000   -4.04e+06    4.04e+06
x1                   0.0259      0.008      3.294      0.001       0.011       0.041
x2                   0.0954      0.022      4.407      0.000       0.053       0.138
x5                   0.0446      0.016      2.826      0.005       0.014       0.076
x7                 -74.1062      1.636    -45.290      0.000     -77.313     -70.899
x8                   0.1096      0.018      6.179      0.000       0.075       0.144
x13                  0.0194      0.009      2.108      0.035       0.001       0.037
x15                 -0.0205      0.014     -1.494      0.135      -0.047       0.006
x18                 -0.0094      0.005     -2.034      0.042      -0.018      -0.000
x19                 -0.0001   1.51e-05     -9.304      0.000      -0.000      -0.000
x20                 -0.0301      0.006     -4.903      0.000      -0.042      -0.018
x21                  0.0012      0.001      1.199      0.231      -0.001       0.003
x25                  0.0058      0.006      0.910      0.363      -0.007       0.018
x26                  0.0143      0.014      1.050      0.294      -0.012       0.041
x27                 -0.0004      0.000     -1.429      0.153      -0.001       0.000
x28                 -0.0973      0.012     -7.867      0.000      -0.122      -0.073
x29                 -0.0202      0.013     -1.574      0.115      -0.045       0.005
x31                 -0.5720      0.059     -9.740      0.000      -0.687      -0.457
x32                  0.0144      0.009      1.642      0.101      -0.003       0.032
x35                  0.0243      0.015      1.670      0.095      -0.004       0.053
x36                 -0.0115      0.010     -1.209      0.227      -0.030       0.007
x37                  0.0623      0.009      6.603      0.000       0.044       0.081
x40                 -0.1068      0.008    -13.138      0.000      -0.123      -0.091
x43                 -0.0168      0.015     -1.118      0.264      -0.046       0.013
x46                 -0.0549      0.012     -4.517      0.000      -0.079      -0.031
x47                 -0.1394      0.008    -16.839      0.000      -0.156      -0.123
x50              -9.406e-05   2.48e-05     -3.796      0.000      -0.000   -4.55e-05
x51                  0.0436      0.010      4.290      0.000       0.024       0.063
x56                 -0.0230      0.011     -2.158      0.031      -0.044      -0.002
x58                  0.0007      0.000      1.596      0.110      -0.000       0.002
x62                 -0.0533      0.013     -3.975      0.000      -0.080      -0.027
x63                  0.0444      0.013      3.485      0.000       0.019       0.069
x66                  0.0689      0.016      4.193      0.000       0.037       0.101
x67                 -0.0254      0.014     -1.867      0.062      -0.052       0.001
x68                  0.0145      0.003      4.265      0.000       0.008       0.021
x70        

In [61]:
pd.DataFrame(results.predict()).head(25)

,0
0,0.040019
1,0.341241
2,0.171202
3,0.450079
4,0.016030
5,0.111500
6,0.041731
7,0.140101
8,0.075196
9,0.145193


### AUC curve on training set
There is no dependent variable in test set.

### Logistic regresion area under curve

In [64]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainLog[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainLog[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainLog[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.7527958790394674
log loss: 0.35991762698348395


### Neural networks area under curve

In [65]:
yTrainClean = dfTrainClean["y"].to_numpy()
print(yTrainClean)
falsePositiveRate,truePositiveRate,threshold = metrics.roc_curve(yTrainClean,yProbTrainNN[:,1])

rocAucScore = metrics.roc_auc_score(yTrainClean,yProbTrainNN[:,1])
print("auc score: %s" % rocAucScore)
logLoss = log_loss(yTrainClean,yProbTrainNN[:,1])
print("log loss: %s" % logLoss)

[0. 1. 1. ... 0. 0. 0.]
auc score: 0.8298681271646436
log loss: 0.31671234359547035
